In [3]:
import pickle
import numpy as np
import pandas as pd
from implicit.bpr import BayesianPersonalizedRanking
from rectools.dataset import Interactions, Dataset
from rectools.model_selection import TimeRangeSplitter
from rectools.metrics import MAP, MeanInvUserFreq, calc_metrics

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel, RandomModel, PopularModel
from tqdm import tqdm

from lightfm import LightFM

In [4]:
interactions_df = pd.read_csv('../artifacts/data_original/interactions.csv')
users = pd.read_csv('../artifacts/data_original/users.csv')
items = pd.read_csv('../artifacts/data_original/items.csv')

interactions_df.rename(columns={'last_watch_dt': Columns.Datetime,
                                'total_dur': Columns.Weight}, inplace=True) 

# Мы хотим учитывать юзеров с 1 или двумя просмотрами, так как этого недостаточно чтобы говорить о пользователях хоть что-нибудь
user_ids_all = interactions_df.groupby(Columns.User)[Columns.Item].nunique().reset_index(name='unique_items_count')
hot_users = user_ids_all[user_ids_all['unique_items_count'] > 10][Columns.User]
interactions_df_hot_users = interactions_df[interactions_df[Columns.User].isin(hot_users)]


users = users[users[Columns.User].isin(interactions_df_hot_users[Columns.User])]
interactions_df_hot_users = interactions_df_hot_users[interactions_df_hot_users[Columns.User].isin(users[Columns.User])]
items = items[items[Columns.Item].isin(interactions_df_hot_users[Columns.Item])]

interactions = Interactions(interactions_df_hot_users)   

interactions.df

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
6,1016458,354,2021-08-14,1672.0,25.0
7,884009,693,2021-08-04,703.0,14.0
12,667487,12173,2021-05-29,96.0,1.0
...,...,...,...,...,...
5476226,435089,13475,2021-07-06,4624.0,85.0
5476239,610017,7107,2021-05-10,1133.0,75.0
5476244,438585,7829,2021-08-02,6804.0,100.0
5476247,546862,9673,2021-04-13,2308.0,49.0


In [5]:
interactions_df_hot_users

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
6,1016458,354,2021-08-14,1672.0,25.0
7,884009,693,2021-08-04,703.0,14.0
12,667487,12173,2021-05-29,96.0,1.0
...,...,...,...,...,...
5476226,435089,13475,2021-07-06,4624.0,85.0
5476239,610017,7107,2021-05-10,1133.0,75.0
5476244,438585,7829,2021-08-02,6804.0,100.0
5476247,546862,9673,2021-04-13,2308.0,49.0


In [6]:
items

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"


In [7]:
N_SPLITS = 1
TEST_SIZE = '7D'

cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=False,
)
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-08-16 00:00:00', freq='7D'),
  Timestamp('2021-08-23 00:00:00', freq='7D'))]

In [8]:
from rectools.metrics import Accuracy, MRR, NDCG

K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 16
N_FACTORS = (32,)
N_EPOCHS = 1 # Lightfm
USER_ALPHA = 0 # Lightfm
ITEM_ALPHA = 0 # Lightfm
LEARNING_RATE = 0.001 # Lightfm

# metrics = {
#     'Precision': Precision(k=10),
#     'Recall': Recall(k=10),
#     'MAP': MAP(k=10),
# }

# metrics = {}
# for metric_name, metric in metrics_name.items():
#     for k in range(5, 10+1):
#         metrics[f'{metric_name}@{k}'] = metric(k=k)

# models = {
#     "random": RandomModel(random_state=RANDOM_STATE),
#     "popular": PopularModel(),
#     "most_raited": PopularModel(popularity="sum_weight"),
# }
# 
# implicit_models = {
#     'ALS': AlternatingLeastSquares,
#     'BPR': BayesianPersonalizedRanking,
# }
# for implicit_name, implicit_model in implicit_models.items():
#     for n_factors in N_FACTORS:
#         models[f"{implicit_name}_{n_factors}"] = ImplicitALSWrapperModel(
#             model=implicit_model(
#                 factors=n_factors, 
#                 random_state=RANDOM_STATE, 
#                 num_threads=NUM_THREADS
#             )
#         )

# lightfm_losses = ('logistic', 'bpr', 'warp')
# 
# for loss in lightfm_losses:
#     for n_factors in N_FACTORS:
#         models[f"LightFM_{loss}_{n_factors}"] = LightFMWrapperModel(
#             LightFM(
#                 no_components=n_factors, 
#                 loss=loss, 
#                 random_state=RANDOM_STATE,
#             ),
#             epochs=10,
#             num_threads=NUM_THREADS,
#         )
# 
models = {
    "random": RandomModel(random_state=42),
    "popular": PopularModel(),
    "most_raited": PopularModel(popularity="sum_weight")
}

metrics = {
    "precision@10": Precision(k=10),
    "accuracy@10": Accuracy(k=10),
    "map@10": MAP(k=10),
    "mrr@10": MRR(k=10),
    "ndcg@10": NDCG(k=10),
    "precision@5": Precision(k=5),
    "accuracy@5": Accuracy(k=5),
    "map@5": MAP(k=5),
    "mrr@5": MRR(k=5),
    "ndcg@5": NDCG(k=5),
    "precision@1": Precision(k=1),
    "accuracy@1": Accuracy(k=1),
    "map@1": MAP(k=1),
    "mrr@1": MRR(k=1),
    "ndcg@1": NDCG(k=1),
}

In [9]:
metrics

{'precision@10': Precision(k=10),
 'accuracy@10': Accuracy(k=10),
 'map@10': MAP(k=10, divide_by_k=False),
 'mrr@10': MRR(k=10),
 'ndcg@10': NDCG(k=10, log_base=2),
 'precision@5': Precision(k=5),
 'accuracy@5': Accuracy(k=5),
 'map@5': MAP(k=5, divide_by_k=False),
 'mrr@5': MRR(k=5),
 'ndcg@5': NDCG(k=5, log_base=2),
 'precision@1': Precision(k=1),
 'accuracy@1': Accuracy(k=1),
 'map@1': MAP(k=1, divide_by_k=False),
 'mrr@1': MRR(k=1),
 'ndcg@1': NDCG(k=1, log_base=2)}

In [10]:
metrics

{'precision@10': Precision(k=10),
 'accuracy@10': Accuracy(k=10),
 'map@10': MAP(k=10, divide_by_k=False),
 'mrr@10': MRR(k=10),
 'ndcg@10': NDCG(k=10, log_base=2),
 'precision@5': Precision(k=5),
 'accuracy@5': Accuracy(k=5),
 'map@5': MAP(k=5, divide_by_k=False),
 'mrr@5': MRR(k=5),
 'ndcg@5': NDCG(k=5, log_base=2),
 'precision@1': Precision(k=1),
 'accuracy@1': Accuracy(k=1),
 'map@1': MAP(k=1, divide_by_k=False),
 'mrr@1': MRR(k=1),
 'ndcg@1': NDCG(k=1, log_base=2)}

In [11]:
models

{'random': <rectools.models.random.RandomModel at 0x7f76abb6b3d0>,
 'popular': <rectools.models.popular.PopularModel at 0x7f76abb6a6b0>,
 'most_raited': <rectools.models.popular.PopularModel at 0x7f76abb6b340>}

In [12]:
models

{'random': <rectools.models.random.RandomModel at 0x7f76abb6b3d0>,
 'popular': <rectools.models.popular.PopularModel at 0x7f76abb6a6b0>,
 'most_raited': <rectools.models.popular.PopularModel at 0x7f76abb6b340>}

In [13]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
1,962099,М,sex
3,721985,Ж,sex
18,155520,М,sex
24,269408,Ж,sex
27,384532,М,sex


In [14]:
users

,user_id,age,income,sex,kids_flg
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
18,155520,age_25_34,income_40_60,М,0
24,269408,age_35_44,income_20_40,Ж,1
27,384532,age_25_34,income_40_60,М,1
...,...,...,...,...,...
840126,815393,age_25_34,income_40_60,М,1
840137,486585,age_25_34,income_20_40,Ж,0
840151,534011,age_25_34,income_20_40,Ж,1
840158,788694,age_35_44,income_60_90,М,0


In [15]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

item_features = pd.concat((genre_feature, content_feature))

In [16]:
%%time
dataset = Dataset.construct(
    interactions_df=interactions_df_hot_users,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

CPU times: user 227 ms, sys: 7.51 ms, total: 234 ms
Wall time: 234 ms


In [17]:
%%time
    
cv.get_test_fold_borders(interactions)

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=cv.n_splits):
    print(f"\n==================== Fold {fold_info['i_split']}")
    print(fold_info)
        
    df_train = interactions.df.iloc[train_ids]
    df_test = interactions.df.iloc[test_ids][Columns.UserItem]
    test_users = np.unique(df_test[Columns.User])
    
    dataset = Dataset.construct(df_train)
    catalog = interactions.df[Columns.Item].unique()

    # dataset = Dataset.construct(
    #     interactions_df=df_train,
    #     user_features_df=user_features,
    #     cat_user_features=["sex", "age", "income"],
    #     item_features_df=item_features,
    #     cat_item_features=["genre", "content_type"],
    # )  

    for model_name, model in models.items():
        print(f"Fitting model {model_name}...")
        model_quality = {'model': model_name}
    
        model.fit(dataset)
        recos = model.recommend(
            users=test_users,
            dataset=dataset,
            k=K_RECOS,
            filter_viewed=True,
        )
        metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=interactions.df,
                catalog=catalog,
            )
        print(metric_values)
        model_quality.update(metric_values)
        results.append(model_quality)
results

  0%|          | 0/1 [00:00<?, ?it/s]


==================== Fold 0
{'i_split': 0, 'start': Timestamp('2021-08-16 00:00:00', freq='7D'), 'end': Timestamp('2021-08-23 00:00:00', freq='7D'), 'train': 2381925, 'train_users': 105793, 'train_items': 14266, 'test': 163151, 'test_users': 41645, 'test_items': 5829}


  0%|          | 0/1 [00:00<?, ?it/s]

Fitting model random...


KeyError: 'All given users must be present in `dataset.user_id_map`'

In [18]:
def get_metrics(models, metrics, cv, k):
    
    cv.get_test_fold_borders(interactions)
    
    results = []
    
    fold_iterator = cv.split(interactions, collect_fold_stats=True)
    
    for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=cv.n_splits):
        print(f"\n==================== Fold {fold_info['i_split']}")
        print(fold_info)
    
        df_train = interactions.df.iloc[train_ids]
        dataset = Dataset.construct(df_train)
    
        df_test = interactions.df.iloc[test_ids][Columns.UserItem]
        test_users = np.unique(df_test[Columns.User])
    
        catalog = df_train[Columns.Item].unique()
    
        for model_name, model in models.items():
            
            model.fit(dataset)
            recos = model.recommend(
                users=test_users,
                dataset=dataset,
                k=k,
                filter_viewed=True,
            )
            
            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )
            print(metric_values)
            res = {"fold": fold_info["i_split"], "model": model_name}
            res.update(metric_values)
            results.append(res)
            
    return results

In [19]:
results = get_metrics(models, metrics, cv, k=10)
results

  0%|          | 0/1 [00:00<?, ?it/s]


==================== Fold 0
{'i_split': 0, 'start': Timestamp('2021-08-16 00:00:00', freq='7D'), 'end': Timestamp('2021-08-23 00:00:00', freq='7D'), 'train': 2381925, 'train_users': 105793, 'train_items': 14266, 'test': 163151, 'test_users': 41645, 'test_items': 5829}


  0%|          | 0/1 [00:00<?, ?it/s]


KeyError: 'All given users must be present in `dataset.user_id_map`'

In [ ]:
df_quality = pd.DataFrame(results).T
df_quality.columns = df_quality.iloc[0]
df_quality.drop('model', inplace=True)

In [ ]:
df_quality.style.highlight_max(color='lightgreen', axis=1).data

In [ ]:
test_users